# 🏠 RECS 2020 Exploratory Data Analysis

## Heat Pump Retrofit Project

**Author:** Fafa (GitHub: Fateme9977)  
**Institution:** K. N. Toosi University of Technology

---

This notebook performs exploratory data analysis on the RECS 2020 microdata
to understand the characteristics of U.S. gas-heated housing stock.

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Project paths
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / 'data'
OUTPUT_DIR = PROJECT_ROOT / 'output'

print(f'Project root: {PROJECT_ROOT}')
print(f'Data directory: {DATA_DIR}')

## 1. Load Raw Data

In [ ]:
# Find the microdata file
csv_files = list(DATA_DIR.glob('recs2020*.csv'))
print(f'Found CSV files: {csv_files}')

if csv_files:
    df_raw = pd.read_csv(csv_files[0], low_memory=False)
    print(f'\nLoaded {len(df_raw):,} households with {len(df_raw.columns)} variables')
else:
    print('No RECS CSV found!')

In [ ]:
# Basic info
print('Dataset shape:', df_raw.shape)
print('\nColumn types:')
print(df_raw.dtypes.value_counts())

In [ ]:
# Display first few rows
df_raw.head()

## 2. Key Variables Overview

In [ ]:
# Key variables for our analysis
key_vars = [
    'DOEID',       # Household ID
    'NWEIGHT',     # Sample weight
    'REGIONC',     # Census region
    'DIVISION',    # Census division
    'HDD65',       # Heating degree days
    'CDD65',       # Cooling degree days
    'TYPEHUQ',     # Housing type
    'YEARMADERANGE', # Year built
    'TOTSQFT_EN',  # Total sqft
    'TOTHSQFT',    # Heated sqft
    'FUELHEAT',    # Main heating fuel
    'EQUIPM',      # Heating equipment
    'DRAFTY',      # Draftiness
    'ADQINSUL',    # Insulation adequacy
    'BTUNG',       # Gas consumption (BTU)
    'BTUSPH',      # Space heating (BTU)
]

# Check which are available
available = [v for v in key_vars if v in df_raw.columns]
missing = [v for v in key_vars if v not in df_raw.columns]

print(f'Available key variables: {len(available)}/{len(key_vars)}')
if missing:
    print(f'Missing: {missing}')

In [ ]:
# Summary statistics for key numeric variables
numeric_vars = ['HDD65', 'CDD65', 'TOTSQFT_EN', 'TOTHSQFT', 'BTUNG', 'BTUSPH', 'NWEIGHT']
numeric_vars = [v for v in numeric_vars if v in df_raw.columns]

df_raw[numeric_vars].describe()

## 3. Heating Fuel Distribution

In [ ]:
# Heating fuel codes
# 1=Natural gas, 2=Propane, 3=Fuel oil, 4=Electricity, 5=Wood, 21=Other

fuel_labels = {
    1: 'Natural Gas',
    2: 'Propane',
    3: 'Fuel Oil',
    4: 'Electricity',
    5: 'Wood',
    21: 'Other'
}

if 'FUELHEAT' in df_raw.columns:
    fuel_counts = df_raw['FUELHEAT'].value_counts()
    fuel_weighted = df_raw.groupby('FUELHEAT')['NWEIGHT'].sum()
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Sample counts
    ax1 = axes[0]
    fuel_counts.rename(index=fuel_labels).plot.bar(ax=ax1, color='steelblue')
    ax1.set_xlabel('Heating Fuel')
    ax1.set_ylabel('Sample Count')
    ax1.set_title('Heating Fuel Distribution (Sample)')
    ax1.tick_params(axis='x', rotation=45)
    
    # Weighted counts
    ax2 = axes[1]
    (fuel_weighted / 1e6).rename(index=fuel_labels).plot.bar(ax=ax2, color='coral')
    ax2.set_xlabel('Heating Fuel')
    ax2.set_ylabel('Households (Millions)')
    ax2.set_title('Heating Fuel Distribution (Weighted)')
    ax2.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Percentages
    print('\nWeighted percentages:')
    print((fuel_weighted / fuel_weighted.sum() * 100).round(1).rename(index=fuel_labels))

## 4. Gas-Heated Homes Analysis

In [ ]:
# Filter for gas-heated homes
df_gas = df_raw[df_raw['FUELHEAT'] == 1].copy()
print(f'Gas-heated homes: {len(df_gas):,} ({len(df_gas)/len(df_raw)*100:.1f}% of sample)')
print(f'Weighted: {df_gas["NWEIGHT"].sum()/1e6:.1f} million households')

In [ ]:
# HDD distribution for gas homes
fig, ax = plt.subplots(figsize=(10, 5))

ax.hist(df_gas['HDD65'], bins=30, weights=df_gas['NWEIGHT']/1e6, 
        color='steelblue', edgecolor='white', alpha=0.7)
ax.set_xlabel('Heating Degree Days (HDD65)', fontsize=12)
ax.set_ylabel('Households (Millions)', fontsize=12)
ax.set_title('Climate Distribution of Gas-Heated Homes', fontsize=14)

# Add climate zone markers
ax.axvline(x=3000, color='green', linestyle='--', label='Mild/Mixed boundary')
ax.axvline(x=6000, color='red', linestyle='--', label='Mixed/Cold boundary')
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Year built distribution
year_labels = {
    1: 'Before 1950',
    2: '1950-1959',
    3: '1960-1969',
    4: '1970-1979',
    5: '1980-1989',
    6: '1990-1999',
    7: '2000-2009',
    8: '2010-2015',
    9: '2016-2020'
}

if 'YEARMADERANGE' in df_gas.columns:
    year_weighted = df_gas.groupby('YEARMADERANGE')['NWEIGHT'].sum() / 1e6
    
    fig, ax = plt.subplots(figsize=(12, 5))
    year_weighted.rename(index=year_labels).plot.bar(ax=ax, color='teal', edgecolor='white')
    ax.set_xlabel('Year Built', fontsize=12)
    ax.set_ylabel('Households (Millions)', fontsize=12)
    ax.set_title('Age Distribution of Gas-Heated Housing Stock', fontsize=14)
    ax.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Draftiness distribution
draft_labels = {
    1: 'Very drafty',
    2: 'Somewhat drafty',
    3: 'Not drafty'
}

if 'DRAFTY' in df_gas.columns:
    draft_weighted = df_gas.groupby('DRAFTY')['NWEIGHT'].sum() / 1e6
    
    fig, ax = plt.subplots(figsize=(8, 5))
    colors = ['#d62728', '#ff7f0e', '#2ca02c']
    draft_weighted.rename(index=draft_labels).plot.bar(ax=ax, color=colors, edgecolor='white')
    ax.set_xlabel('Draftiness Level', fontsize=12)
    ax.set_ylabel('Households (Millions)', fontsize=12)
    ax.set_title('Envelope Draftiness in Gas-Heated Homes', fontsize=14)
    ax.tick_params(axis='x', rotation=0)
    
    plt.tight_layout()
    plt.show()

## 5. Geographic Distribution

In [ ]:
# Census division distribution
division_labels = {
    1: 'New England',
    2: 'Middle Atlantic',
    3: 'East North Central',
    4: 'West North Central',
    5: 'South Atlantic',
    6: 'East South Central',
    7: 'West South Central',
    8: 'Mountain',
    9: 'Pacific',
}

if 'DIVISION' in df_gas.columns:
    div_weighted = df_gas.groupby('DIVISION')['NWEIGHT'].sum() / 1e6
    div_hdd = df_gas.groupby('DIVISION')['HDD65'].mean()
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Household counts
    ax1 = axes[0]
    div_weighted.rename(index=division_labels).sort_values(ascending=True).plot.barh(
        ax=ax1, color='steelblue', edgecolor='white'
    )
    ax1.set_xlabel('Households (Millions)', fontsize=12)
    ax1.set_title('Gas-Heated Homes by Census Division', fontsize=14)
    
    # Mean HDD
    ax2 = axes[1]
    div_hdd.rename(index=division_labels).sort_values(ascending=True).plot.barh(
        ax=ax2, color='coral', edgecolor='white'
    )
    ax2.set_xlabel('Mean HDD65', fontsize=12)
    ax2.set_title('Mean Heating Degree Days by Division', fontsize=14)
    
    plt.tight_layout()
    plt.show()

## 6. Heating Consumption Analysis

In [ ]:
# Calculate thermal intensity
if 'TOTHSQFT' in df_gas.columns and 'BTUSPH' in df_gas.columns:
    df_gas = df_gas[
        (df_gas['TOTHSQFT'] > 0) & 
        (df_gas['HDD65'] > 0) & 
        (df_gas['BTUSPH'] > 0)
    ].copy()
    
    df_gas['Thermal_Intensity'] = df_gas['BTUSPH'] / (df_gas['TOTHSQFT'] * df_gas['HDD65'])
    
    print('Thermal Intensity Statistics (BTU/sqft/HDD):')
    print(df_gas['Thermal_Intensity'].describe())

In [ ]:
# Distribution of thermal intensity
if 'Thermal_Intensity' in df_gas.columns:
    fig, ax = plt.subplots(figsize=(10, 5))
    
    # Remove extreme outliers for visualization
    intensity_clean = df_gas['Thermal_Intensity'][
        df_gas['Thermal_Intensity'] < df_gas['Thermal_Intensity'].quantile(0.99)
    ]
    
    ax.hist(intensity_clean, bins=50, color='teal', edgecolor='white', alpha=0.7)
    ax.axvline(x=intensity_clean.median(), color='red', linestyle='--', 
               label=f'Median: {intensity_clean.median():.2f}')
    ax.axvline(x=intensity_clean.mean(), color='orange', linestyle='--',
               label=f'Mean: {intensity_clean.mean():.2f}')
    
    ax.set_xlabel('Thermal Intensity (BTU/sqft/HDD)', fontsize=12)
    ax.set_ylabel('Count', fontsize=12)
    ax.set_title('Distribution of Heating Thermal Intensity', fontsize=14)
    ax.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Thermal intensity by draftiness
if 'Thermal_Intensity' in df_gas.columns and 'DRAFTY' in df_gas.columns:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    df_plot = df_gas[df_gas['Thermal_Intensity'] < df_gas['Thermal_Intensity'].quantile(0.99)]
    
    sns.boxplot(
        data=df_plot,
        x='DRAFTY',
        y='Thermal_Intensity',
        palette=['#d62728', '#ff7f0e', '#2ca02c'],
        ax=ax
    )
    
    ax.set_xticklabels(['Very Drafty', 'Somewhat Drafty', 'Not Drafty'])
    ax.set_xlabel('Draftiness Level', fontsize=12)
    ax.set_ylabel('Thermal Intensity (BTU/sqft/HDD)', fontsize=12)
    ax.set_title('Thermal Intensity by Envelope Quality', fontsize=14)
    
    plt.tight_layout()
    plt.show()

## 7. Correlation Analysis

In [ ]:
# Correlation matrix for key variables
corr_vars = ['HDD65', 'TOTHSQFT', 'YEARMADERANGE', 'DRAFTY', 'ADQINSUL', 'BTUSPH', 'Thermal_Intensity']
corr_vars = [v for v in corr_vars if v in df_gas.columns]

if len(corr_vars) > 2:
    corr_matrix = df_gas[corr_vars].corr()
    
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(
        corr_matrix,
        annot=True,
        cmap='RdBu_r',
        center=0,
        fmt='.2f',
        ax=ax
    )
    ax.set_title('Correlation Matrix of Key Variables', fontsize=14)
    
    plt.tight_layout()
    plt.show()

## 8. Summary and Next Steps

### Key Findings:
1. Natural gas is the dominant heating fuel, covering ~48% of U.S. households
2. Gas-heated homes span a wide range of climates (HDD: 1,000-9,000+)
3. A significant portion of stock is pre-1980 with envelope quality issues
4. Thermal intensity varies significantly with envelope quality

### Next Steps:
1. Run `01_data_prep.py` to create cleaned analytical dataset
2. Run `02_descriptive_validation.py` to validate against official tables
3. Train XGBoost model (`03_xgboost_model.py`)
4. Perform SHAP analysis (`04_shap_analysis.py`)

In [ ]:
# Save summary statistics
summary = {
    'total_households': len(df_raw),
    'gas_heated_households': len(df_gas),
    'gas_heated_pct': len(df_gas) / len(df_raw) * 100,
    'weighted_gas_millions': df_gas['NWEIGHT'].sum() / 1e6,
    'mean_hdd': df_gas['HDD65'].mean(),
    'mean_sqft': df_gas['TOTHSQFT'].mean() if 'TOTHSQFT' in df_gas.columns else None,
    'mean_thermal_intensity': df_gas['Thermal_Intensity'].mean() if 'Thermal_Intensity' in df_gas.columns else None,
}

print('\n=== EDA Summary ===')
for k, v in summary.items():
    if v is not None:
        print(f'{k}: {v:.2f}' if isinstance(v, float) else f'{k}: {v}')